In [1]:
import os
os.chdir('/Users/layne/Desktop/aiqc')
os.getcwd()

'/Users/layne/Desktop/aiqc'

In [2]:
import aiqc
from aiqc import datum

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)



=> Welcome to AIQC.
To get started, run `aiqc.create_folder()` followed by `aiqc.create_config()`.


=> Info - Cannot fetch database yet because it has not been configured.



In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import *

In [4]:
aiqc.create_folder()
aiqc.create_config()
aiqc.create_db()

# from importlib import reload
# aiqc.delete_db(True)
# reload(aiqc)
# aiqc.create_db()

=> Info - it appears the following folder does not exist on your system:
/Users/layne/Library/Application Support/aiqc/

=> Fix - you can attempt to fix this by running `aiqc.create_folder()`.

=> Success - created folder at file path:
/Users/layne/Library/Application Support/aiqc/

=> Next try running `aiqc.create_config()`.


=> Success - the following file path already exists on your system:
/Users/layne/Library/Application Support/aiqc/


=> Success - created config file for settings at path:
/Users/layne/Library/Application Support/aiqc/config.json


=> Success - created database file at path:
/Users/layne/Library/Application Support/aiqc/aiqc.sqlite3


=> Success - created the following tables within database:
['algorithm', 'batch', 'dataset', 'encoderset', 'featurecoder', 'featureset', 'file', 'fold', 'foldset', 'hyperparamcombo', 'hyperparamset', 'image', 'job', 'jobset', 'label', 'labelcoder', 'result', 'splitset', 'tabular']



---

In [5]:
dataset = aiqc.Dataset.Tabular.from_path(
    examples.get_demo_file_path('exoplanets.parquet')
    , source_file_format = 'parquet'
)

In [6]:
dataset.to_pandas().head().dtypes

TypeFlag                  int64
PlanetaryMassJpt        float64
PeriodDays              float64
SurfaceTempK            float64
DistFromSunParsec       float64
HostStarMassSlrMass     float64
HostStarRadiusSlrRad    float64
HostStarMetallicity     float64
HostStarTempK           float64
dtype: object

In [7]:
label = dataset.make_label(columns=['SurfaceTempK'])

In [8]:
featureset = dataset.make_featureset(exclude_columns=[
    'SurfaceTempK',
])

In [9]:
featureset.get_dtypes()

{'TypeFlag': 'int64',
 'PlanetaryMassJpt': 'float64',
 'PeriodDays': 'float64',
 'DistFromSunParsec': 'float64',
 'HostStarMassSlrMass': 'float64',
 'HostStarRadiusSlrRad': 'float64',
 'HostStarMetallicity': 'float64',
 'HostStarTempK': 'float64'}

In [10]:
splitset = featureset.make_splitset(
    label_id=label.id
    , size_test = 0.22
    , size_validation = 0.12 
)

In [11]:
foldset = splitset.make_foldset(fold_count=3)

---

In [12]:
encoderset = splitset.make_encoderset()

In [13]:
labelcoder = encoderset.make_labelcoder(
    sklearn_preprocess = StandardScaler(copy=False)
)

---
---
---

In [14]:
featurecoder_kwargs = [
    
    dict(# OneHotEncode the integers.
        sklearn_preprocess = OneHotEncoder(sparse=False),
        dtypes = ['int64']),

    dict(# Scale the decimals.
        sklearn_preprocess = StandardScaler(copy=False),
        dtypes = ['float64'])
]

In [15]:
for fc in featurecoder_kwargs:
    encoderset.make_featurecoder(**fc)


=> The columns below matched your filters. Tested encoding them successfully.

['TypeFlag']


=> The remaining columns and dtypes can be used in your downstream Featurecoder:

{'DistFromSunParsec': 'float64',
 'HostStarMassSlrMass': 'float64',
 'HostStarMetallicity': 'float64',
 'HostStarRadiusSlrRad': 'float64',
 'HostStarTempK': 'float64',
 'PeriodDays': 'float64',
 'PlanetaryMassJpt': 'float64'}


=> The columns below matched your filters. Tested encoding them successfully.

['PlanetaryMassJpt',
 'PeriodDays',
 'DistFromSunParsec',
 'HostStarMassSlrMass',
 'HostStarRadiusSlrRad',
 'HostStarMetallicity',
 'HostStarTempK']


=> Nice! Now all columns have encoders associated with them. No more Featurecoders can be added to this Encoderset.



---
---
---

In [22]:
import keras
from keras import metrics
from keras.models import Sequential
from keras.callbacks import History
from keras.layers import Dense, Dropout

In [24]:
def function_model_build(input_shape, **hyperparameters):
    model = Sequential()
    model.add(Dense(hyperparameters['neuron_count'], input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(hyperparameters['neuron_count'], activation='relu'))
    
    if hyperparameters['2nd_dropout']:
        model.add(Dropout(0.2))
        
    model.add(Dense(1, kernel_initializer='normal'))
    
    
    opt = keras.optimizers.Adamax(hyperparameters['learning_rate'])
    model.compile(
		loss='mean_squared_error'
		, optimizer=opt
	)
    return model

In [25]:
def function_model_train(model, samples_train, samples_evaluate, **hyperparameters):   
    
    metrics_cuttoffs = [
        {"metric":"val_loss", "cutoff":0.3, "above_or_below":"below"},
        {"metric":"loss", "cutoff":0.3, "above_or_below":"below"}
    ]
    cutoffs = aiqc.TrainingCallback.Keras.MetricCutoff(metrics_cuttoffs)
    
    model.fit(
        samples_train["features"]
        , samples_train["labels"]
        , validation_data = (
            samples_evaluate["features"]
            , samples_evaluate["labels"]
        )
        , verbose = 0
        , batch_size = hyperparameters['batch_size']
        , callbacks=[History(), cutoffs]
        , epochs = hyperparameters['epoch_count']
    )

    return model

In [26]:
algorithm = aiqc.Algorithm.make(
    library = "keras"
    , analysis_type = "regression"
    , function_model_build = function_model_build
    , function_model_train = function_model_train
)

In [27]:
hyperparameters = {
    "batch_size": [3, 5]
    , "2nd_dropout": [True, False]
    , "epoch_count": [500]
    , "neuron_count": [32, 64]
    , "learning_rate": [0.01]
}

In [28]:
hyperparamset = aiqc.Hyperparamset.from_algorithm(
    algorithm_id = algorithm.id
    , hyperparameters = hyperparameters
)

In [29]:
batch = aiqc.Batch.from_algorithm(
    algorithm_id = algorithm.id
    , splitset_id = splitset.id
    , hyperparamset_id = hyperparamset.id
    , foldset_id = None
    , encoderset_id = encoderset.id
)

In [30]:
batch.run_jobs()

🔮 Training Models 🔮:  12%|█████▎                                    | 1/8 [00:37<04:20, 37.28s/it]


:: Epoch #97 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  25%|██████████▌                               | 2/8 [00:46<02:52, 28.78s/it]


:: Epoch #67 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  50%|█████████████████████                     | 4/8 [01:29<01:46, 26.53s/it]


:: Epoch #118 ::
Satisfied thresholds defined in `MetricCutoff` callback. Stopped training early.



🔮 Training Models 🔮:  62%|██████████████████████████▎               | 5/8 [01:42<01:01, 20.45s/it]
Process aiqc_batch_1:
Traceback (most recent call last):
  File "/Users/layne/.pyenv/versions/3.7.6/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/layne/.pyenv/versions/3.7.6/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/layne/Desktop/aiqc/aiqc/__init__.py", line 3234, in background_proc
    job.run(verbose=verbose, repeat_index=repeat_index)
  File "/Users/layne/Desktop/aiqc/aiqc/__init__.py", line 3786, in run
    , **hyperparameters
  File "<ipython-input-25-ea8b7976d0ab>", line 19, in function_model_train
    , epochs = hyperparameters['epoch_count']
  File "/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 108, in _method_wrapper
    return method(self, *args, **kwargs)
  File "/Users

In [17]:
import textwrap